In [1]:
%%capture
!pip install --upgrade -tensorflow_hub
# !pip install -U -huggingface_hub

import textattack
import transformers
import torch
import time
from datasets import Dataset
import sys
import hashlib
import numpy as np

from transformers import AutoTokenizer, AutoModelForSequenceClassification, BertForMaskedLM, pipeline
from textattack.attack_recipes import (
    TextBuggerLi2018, DeepWordBugGao2018, TextFoolerJin2019, BERTAttackLi2020
)
from textattack.constraints.semantics.sentence_encoders import UniversalSentenceEncoder
from textattack.models.wrappers import ModelWrapper

sys.path.append('../')
from eval_utils import *
sys.path.pop()

2023-08-02 17:16:04.963402: I tensorflow/core/platform/cpu_feature_guard.cc:182] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.
2023-08-02 17:16:06.233875: W tensorflow/compiler/tf2tensorrt/utils/py_utils.cc:38] TF-TRT Warning: Could not find TensorRT


In [3]:
# set a seed, because reproducability is cool
np.random.seed(int(hashlib.sha256('Harrison Gietz'.encode('utf-8')).hexdigest(), 16) % 2**32)
torch.cuda.empty_cache()

device = input('enter a device name to run on: ')
dataset_val = input('Enter the number of samples to run on (100 or 1000): ')
defense = input('Specify a defense type among "default", "logit", "maj_log", "one_hot": ')
cand_size = int(input('enter number of candidates (recommended 12 for quicker run, 50 otherwise): '))

ag_tokenizer = AutoTokenizer.from_pretrained("textattack/bert-base-uncased-ag-news")
ag_model = AutoModelForSequenceClassification.from_pretrained("textattack/bert-base-uncased-ag-news")
ag_model.to(device)
ag_pipeline = pipeline('sentiment-analysis', model=ag_model, tokenizer=ag_tokenizer)
ag_pipeline.device = next(ag_model.parameters()).device

ag_model_directory = "../../../models/bert-uncased_maskedlm_agnews_july31" #first diff
finetuned_ag_maskedlm = BertForMaskedLM.from_pretrained(ag_model_directory)
finetuned_ag_maskedlm.to(device)
ag_fill_mask = pipeline("fill-mask", model=finetuned_ag_maskedlm, tokenizer=ag_tokenizer)
ag_fill_mask.device = next(ag_model.parameters()).device

num_voter = 11
mask_pct = 0.3    
    
attack = TextFoolerJin2019

if dataset_val == '100':
    loaded_ag_100 = Dataset.load_from_disk('../data/filtered_ag_clean_100')
    ag_100 = textattack.datasets.Dataset(convert_to_tuples(loaded_ag_100))
    dataset = ag_100
    dataset_name = 'ag-news100'
elif dataset_val =='1000':
    loaded_ag_1000 = Dataset.load_from_disk('../data/filtered_ag_clean_1000')
    ag_1000 = textattack.datasets.Dataset(convert_to_tuples(loaded_ag_1000))
    dataset = ag_1000
    dataset_name = 'ag-news1000'
else:
    raise ValueError('Number of samples not supported')
    
if defense == "default":
    ag_wrapper = textattack.models.wrappers.HuggingFaceModelWrapper(ag_model, ag_tokenizer)
    print(ag_wrapper)
elif defense == "logit":
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'logit')
elif defense == 'maj_log':
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'maj_log')
elif defense == "one_hot":
    ag_wrapper = MaskDemaskWrapper(ag_model, ag_tokenizer, ag_fill_mask, num_voter, mask_pct, 'maj_one_hot')
else:
    raise ValueError('Not a valid defense type.')
    
print(f'using num_voter = {num_voter} and mask_pct = {mask_pct} with dataset = {dataset_name}...')

# Parse the attack name
attack_name = parse_attack_name(attack)
attack = attack.build(ag_wrapper)


# change candidate size
attack.transformation.max_candidates = cand_size
# adjust attack threshold to match Li et al. 2023 (0.5 theshold for AgNews Universal sentences encoder):
attack.constraints[2] = UniversalSentenceEncoder(metric = 'angular', threshold = 0.5, 
                                                 window_size = 15, skip_text_shorter_than_window=True, 
                                                 compare_against_original=False)

# Set up arguments for the attack
attack_args = textattack.AttackArgs(
    num_examples=len(dataset),
    log_to_csv=f'{attack_name}_{dataset_name}_candsize{cand_size}_mp{mask_pct}_nv{num_voter}_{defense}_log.csv',
    checkpoint_interval=25, 
    checkpoint_dir="chkpts_2", 
    disable_stdout=True
)
# Perform the attack and save the results
attacker = textattack.Attacker(attack, dataset, attack_args)
attacker.attack_dataset()

print(f'The above are results for {attack_name}_{dataset_name}_candsize{cand_size}_mp{mask_pct}_nv{num_voter}_{defense}.')

enter a device name to run on: cuda:1
Enter the number of samples to run on (100 or 1000): 1000
Specify a defense type among "default", "logit", "maj_log", "one_hot": logit
enter number of candidates (recommended 12 for quicker run, 50 otherwise): 12
using num_voter = 11 and mask_pct = 0.3 with dataset = ag-news1000...
Attack(
  (search_method): GreedyWordSwapWIR(
    (wir_method):  delete
  )
  (goal_function):  UntargetedClassification
  (transformation):  WordSwapEmbedding(
    (max_candidates):  12
    (embedding):  WordEmbedding
  )
  (constraints): 
    (0): WordEmbeddingDistance(
        (embedding):  WordEmbedding
        (min_cos_sim):  0.5
        (cased):  False
        (include_unknown_words):  True
        (compare_against_original):  True
      )
    (1): PartOfSpeech(
        (tagger_type):  nltk
        (tagset):  universal
        (allow_verb_noun_swap):  True
        (compare_against_original):  True
      )
    (2): UniversalSentenceEncoder(
        (metric):  angula


  0%|          | 0/1000 [00:00<?, ?it/s]/usr/local/lib/python3.8/dist-packages/transformers/pipelines/base.py:1081: UserWarning: You seem to be using the pipelines sequentially on GPU. In order to maximize efficiency please use a dataset
  warnings.warn(
2023-08-02 17:17:35.419185: W tensorflow/core/common_runtime/gpu/gpu_device.cc:1956] Cannot dlopen some GPU libraries. Please make sure the missing libraries mentioned above are installed properly if you would like to use GPU. Follow the guide at https://www.tensorflow.org/install/gpu for how to download and setup the required libraries for your platform.
Skipping registering GPU devices...
2023-08-02 17:17:38.079115: I tensorflow/core/common_runtime/executor.cc:1197] [/device:CPU:0] (DEBUG INFO) Executor start aborting (this does not indicate an error and you can ignore this message): INVALID_ARGUMENT: You must feed a value for placeholder tensor 'inputs' with dtype string
	 [[{{node inputs}}]]

  0%|          | 1/1000 [02:35<43:10:2


[Succeeded / Failed / Skipped / Total] 8 / 16 / 1 / 25:   3%|▎         | 26/1000 [51:49<32:21:27, 119.60s/it]
[Succeeded / Failed / Skipped / Total] 8 / 17 / 1 / 26:   3%|▎         | 26/1000 [51:49<32:21:28, 119.60s/it]
[Succeeded / Failed / Skipped / Total] 8 / 17 / 1 / 26:   3%|▎         | 27/1000 [52:19<31:25:23, 116.26s/it]
[Succeeded / Failed / Skipped / Total] 9 / 17 / 1 / 27:   3%|▎         | 27/1000 [52:19<31:25:24, 116.26s/it]
[Succeeded / Failed / Skipped / Total] 9 / 17 / 1 / 27:   3%|▎         | 28/1000 [53:27<30:55:34, 114.54s/it]
[Succeeded / Failed / Skipped / Total] 10 / 17 / 1 / 28:   3%|▎         | 28/1000 [53:27<30:55:34, 114.54s/it]
[Succeeded / Failed / Skipped / Total] 10 / 17 / 1 / 28:   3%|▎         | 29/1000 [53:27<29:50:04, 110.61s/it]
[Succeeded / Failed / Skipped / Total] 10 / 17 / 2 / 29:   3%|▎         | 29/1000 [53:27<29:50:04, 110.61s/it]
[Succeeded / Failed / Skipped / Total] 10 / 17 / 2 / 29:   3%|▎         | 30/1000 [54:53<29:34:47, 109.78s/it]
[Succ


[Succeeded / Failed / Skipped / Total] 17 / 30 / 3 / 50:   5%|▌         | 51/1000 [1:24:06<26:05:12, 98.96s/it]
[Succeeded / Failed / Skipped / Total] 18 / 30 / 3 / 51:   5%|▌         | 51/1000 [1:24:06<26:05:12, 98.96s/it]
[Succeeded / Failed / Skipped / Total] 18 / 30 / 3 / 51:   5%|▌         | 52/1000 [1:26:35<26:18:39, 99.92s/it]
[Succeeded / Failed / Skipped / Total] 18 / 31 / 3 / 52:   5%|▌         | 52/1000 [1:26:35<26:18:40, 99.92s/it]
[Succeeded / Failed / Skipped / Total] 18 / 31 / 3 / 52:   5%|▌         | 53/1000 [1:28:34<26:22:45, 100.28s/it]
[Succeeded / Failed / Skipped / Total] 18 / 32 / 3 / 53:   5%|▌         | 53/1000 [1:28:34<26:22:45, 100.28s/it]
[Succeeded / Failed / Skipped / Total] 18 / 32 / 3 / 53:   5%|▌         | 54/1000 [1:30:39<26:28:14, 100.73s/it]
[Succeeded / Failed / Skipped / Total] 18 / 33 / 3 / 54:   5%|▌         | 54/1000 [1:30:39<26:28:14, 100.73s/it]
[Succeeded / Failed / Skipped / Total] 18 / 33 / 3 / 54:   6%|▌         | 55/1000 [1:32:47<26:34:25


[Succeeded / Failed / Skipped / Total] 28 / 42 / 5 / 75:   8%|▊         | 76/1000 [1:58:22<23:59:07, 93.45s/it]
[Succeeded / Failed / Skipped / Total] 29 / 42 / 5 / 76:   8%|▊         | 76/1000 [1:58:22<23:59:08, 93.45s/it]
[Succeeded / Failed / Skipped / Total] 29 / 42 / 5 / 76:   8%|▊         | 77/1000 [2:00:05<23:59:27, 93.57s/it]
[Succeeded / Failed / Skipped / Total] 29 / 43 / 5 / 77:   8%|▊         | 77/1000 [2:00:05<23:59:27, 93.57s/it]
[Succeeded / Failed / Skipped / Total] 29 / 43 / 5 / 77:   8%|▊         | 78/1000 [2:01:49<23:59:59, 93.71s/it]
[Succeeded / Failed / Skipped / Total] 30 / 43 / 5 / 78:   8%|▊         | 78/1000 [2:01:49<23:59:59, 93.71s/it]
[Succeeded / Failed / Skipped / Total] 30 / 43 / 5 / 78:   8%|▊         | 79/1000 [2:03:21<23:58:13, 93.70s/it]
[Succeeded / Failed / Skipped / Total] 30 / 44 / 5 / 79:   8%|▊         | 79/1000 [2:03:21<23:58:13, 93.70s/it]
[Succeeded / Failed / Skipped / Total] 30 / 44 / 5 / 79:   8%|▊         | 80/1000 [2:05:19<24:01:13, 93

...


[Succeeded / Failed / Skipped / Total] 299 / 588 / 38 / 925:  93%|█████████▎| 926/1000 [25:27:08<2:02:02, 98.95s/it]
[Succeeded / Failed / Skipped / Total] 300 / 588 / 38 / 926:  93%|█████████▎| 926/1000 [25:27:08<2:02:02, 98.95s/it]
[Succeeded / Failed / Skipped / Total] 300 / 588 / 38 / 926:  93%|█████████▎| 927/1000 [25:29:32<2:00:26, 99.00s/it]
[Succeeded / Failed / Skipped / Total] 300 / 589 / 38 / 927:  93%|█████████▎| 927/1000 [25:29:32<2:00:26, 99.00s/it]
[Succeeded / Failed / Skipped / Total] 300 / 589 / 38 / 927:  93%|█████████▎| 928/1000 [25:31:40<1:58:50, 99.03s/it]
[Succeeded / Failed / Skipped / Total] 300 / 590 / 38 / 928:  93%|█████████▎| 928/1000 [25:31:40<1:58:50, 99.03s/it]
[Succeeded / Failed / Skipped / Total] 300 / 590 / 38 / 928:  93%|█████████▎| 929/1000 [25:33:24<1:57:11, 99.04s/it]
[Succeeded / Failed / Skipped / Total] 300 / 591 / 38 / 929:  93%|█████████▎| 929/1000 [25:33:24<1:57:11, 99.04s/it]
[Succeeded / Failed / Skipped / Total] 300 / 591 / 38 / 929:  9


[Succeeded / Failed / Skipped / Total] 306 / 605 / 39 / 950:  95%|█████████▌| 951/1000 [26:12:41<1:21:01, 99.22s/it]
[Succeeded / Failed / Skipped / Total] 306 / 606 / 39 / 951:  95%|█████████▌| 951/1000 [26:12:41<1:21:01, 99.22s/it]
[Succeeded / Failed / Skipped / Total] 306 / 606 / 39 / 951:  95%|█████████▌| 952/1000 [26:15:47<1:19:27, 99.31s/it]
[Succeeded / Failed / Skipped / Total] 306 / 607 / 39 / 952:  95%|█████████▌| 952/1000 [26:15:47<1:19:27, 99.31s/it]
[Succeeded / Failed / Skipped / Total] 306 / 607 / 39 / 952:  95%|█████████▌| 953/1000 [26:17:13<1:17:47, 99.30s/it]
[Succeeded / Failed / Skipped / Total] 306 / 608 / 39 / 953:  95%|█████████▌| 953/1000 [26:17:13<1:17:47, 99.30s/it]
[Succeeded / Failed / Skipped / Total] 306 / 608 / 39 / 953:  95%|█████████▌| 954/1000 [26:17:45<1:16:04, 99.23s/it]
[Succeeded / Failed / Skipped / Total] 307 / 608 / 39 / 954:  95%|█████████▌| 954/1000 [26:17:45<1:16:04, 99.23s/it]
[Succeeded / Failed / Skipped / Total] 307 / 608 / 39 / 954:  9


[Succeeded / Failed / Skipped / Total] 312 / 623 / 40 / 975:  98%|█████████▊| 976/1000 [26:55:05<39:42, 99.29s/it]
[Succeeded / Failed / Skipped / Total] 312 / 624 / 40 / 976:  98%|█████████▊| 976/1000 [26:55:05<39:42, 99.29s/it]
[Succeeded / Failed / Skipped / Total] 312 / 624 / 40 / 976:  98%|█████████▊| 977/1000 [26:56:51<38:03, 99.30s/it]
[Succeeded / Failed / Skipped / Total] 313 / 624 / 40 / 977:  98%|█████████▊| 977/1000 [26:56:51<38:03, 99.30s/it]
[Succeeded / Failed / Skipped / Total] 313 / 624 / 40 / 977:  98%|█████████▊| 978/1000 [26:59:05<36:25, 99.33s/it]
[Succeeded / Failed / Skipped / Total] 313 / 625 / 40 / 978:  98%|█████████▊| 978/1000 [26:59:05<36:25, 99.33s/it]
[Succeeded / Failed / Skipped / Total] 313 / 625 / 40 / 978:  98%|█████████▊| 979/1000 [26:59:06<34:43, 99.23s/it]
[Succeeded / Failed / Skipped / Total] 313 / 625 / 41 / 979:  98%|█████████▊| 979/1000 [26:59:06<34:43, 99.23s/it]
[Succeeded / Failed / Skipped / Total] 313 / 625 / 41 / 979:  98%|█████████▊| 9

[Succeeded / Failed / Skipped / Total] 320 / 639 / 41 / 1000: 100%|██████████| 1000/1000 [27:38:05<00:00, 99.49s/it]


+-------------------------------+--------+
| Attack Results                |        |
+-------------------------------+--------+
| Number of successful attacks: | 320    |
| Number of failed attacks:     | 639    |
| Number of skipped attacks:    | 41     |
| Original accuracy:            | 95.9%  |
| Accuracy under attack:        | 63.9%  |
| Attack success rate:          | 33.37% |
| Average perturbed word %:     | 14.63% |
| Average num. words per input: | 38.43  |
| Avg num queries:              | 209.27 |
+-------------------------------+--------+
The above are results for TextFoolerJin2019_ag-news1000_candsize12_mp0.3_nv11_logit.
